In [ ]:
# Find out if doing random turns can cover a rectangle

import math
import random
import plotly.express as px
import pandas as pd
from shapely.geometry import LineString

In [ ]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("<style>.output_result { max-width:100% !important; }</style>"))
display(HTML("<style>.prompt { display:none !important; }</style>"))

In [ ]:
# Rectangle depcting the room
# Starting point is minx, miny
minx = 0.0
miny = 0.0
maxx = 15.0
maxy = 10.0

# List of rectangular objects in the room, list of (x0, y0, x1, y1)
rectangular_objects = [(3, 4.7, 3.5, 5.2), (4, 4.7, 4.5, 5.2), # Two legs of table
                       (2.8, 4, 3, 4.2), (3.7, 4, 3.9, 4.2), (4.5, 4, 4.7, 4.2), # 1st seating row
                       (2.8, 5.8, 3, 6), (3.7, 5.8, 3.9, 6), (4.5, 5.8, 4.7, 6), # 2nd seating row
                       (1, 1, 1.3, 1.3), (1, 9, 1.3, 9.3), (14, 1, 14.3, 1.3), (14, 9, 14.3, 9.3), # 4 corner tables
                       (9, 3, 10, 7) # Couch
                      ]

# Empty room
# rectangular_objects = []

def hit_any_obj(px0, py0, px1, py1):
    path = LineString([(px0, py0), (px1, py1)])
    for (x0, y0, x1, y1) in rectangular_objects:
        # Just an approximation to check the intersection with one diagonal
        # instead of all 4 sides and try to figure out the intersection point,
        # if the path intersects multiple edges.
        edge = LineString([(x0, y0), (x1, y1)])
        if path.intersects(edge):
            p = path.intersection(edge)
            # Move a bit off from intersection point, else it will be stuck there.
            # This will be valid in read world scenario as well.
            return (p.x + 0.01, p.y + 0.01)

    return None

# Starting angle is 0 to max_angle in degrees
max_angle = 360 #90.0

def get_initial_angle():
    return 0 + random.random() * max_angle

# Drift angle in degrees after travelling distance or
# turning around after hitting a wall
min_drift = -5.0
max_drift = 5.0

def get_drift():
    return min_drift + random.random() * (max_drift - min_drift)

# How much distance robot travels before drifting
min_dist = 0.1*maxx
max_dist = 0.4*maxx

def get_dist():
    return min_dist + random.random() * (max_dist - min_dist)

# Move distance d from point (x, y) at 'angle'. If goes beyond the wall,
# stop at the wall, and change the angle for next move.
def move(x, y, angle):
    # current position is (x, y), and heading at an angle
    d = get_dist() # dist travelled
    dangle = get_drift()
    angle += dangle
    
    newx = x + d*math.cos(math.radians(angle))
    newy = y + d*math.sin(math.radians(angle))
    
    hit_obj = hit_any_obj(x, y, newx, newy)
    if hit_obj:
        # print("Hit one object")
        newx = hit_obj[0]
        newy = hit_obj[1]
    
    hit_the_wall = False
    
    if newx > maxx or newx < minx:
        hit_the_wall = True
        newx = maxx if newx > maxx else minx
        newy = y + (newx - x) * math.tan(math.radians(angle))    
    
    if newy > maxy or newy < miny:
        hit_the_wall = True
        newy = maxy if newy > maxy else miny
        newx = x + (newy - y) / math.tan(math.radians(angle))
    
    if hit_the_wall or hit_obj:
        angle = get_initial_angle()

    # Did the line segment intersect any of the objects in the room?
    # If yes, get the intersection point
    
    return (newx, newy, angle)

In [ ]:
# Number of moves to simulate
def get_points(moves=100):
    # Start of program
    x = maxx/2
    y = maxy/2
    angle = get_initial_angle()

    xs = []
    ys = []

    for i in range(moves):
        xs.append(x)
        ys.append(y)
        #print(x, y)
        (x, y, angle) = move(x, y, angle)

    return (xs, ys)

In [ ]:
def plot_room(xs, ys, rects, lwidth=20):
    # print(xs)
    # print(ys)
    df = pd.DataFrame(dict(
        x = xs, # [1.1, 3, 2, 4],
        y = ys # [1, 2.333, 3, 4]
    ))
    fig = px.line(df, x="x", y="y", title="Robot Moves")
    fig.update_traces(line=dict(width=lwidth))
    fig.add_hrect(y0=miny, y1=maxy)
    #fig.add_vrect(x0=minx, x1=maxx)
    
    # Add objects in the room
    for (x0, y0, x1, y1) in rects:
        fig.add_shape(type="rect",
            xref="x", yref="y",
            x0=x0, y0=y0,
            x1=x1, y1=y1,
            line=dict(
                color="Red",
                #width=3,
            ),
            # fillcolor="LightSkyBlue",
        )

    
    fig.update_layout(xaxis_range=[minx, maxx], yaxis_range=[miny, maxy], width=1200, height=800)
    fig.show()

In [ ]:
(xs, ys) = get_points(100)
# plot_room(xs, ys, [], 1.5)

plot_room(xs, ys, rectangular_objects, 1.5)

plot_room(xs, ys, rectangular_objects, 30)